# The DMOP Files

Please ensure you have LightGBM installed for this code to work.

## EDA

In [ ]:
## use the following command for installing lightGBM, an ML model good at dealing with high cardinality categorical variables

# !pip install lightgbm

In [ ]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

pd.set_option('display.max_rows', 500)

DMOP stands for "Detailed Mission Operations Plan". According to the 2nd place submission in the competition, the DMOP file has the greatest effect on the temperature of the spacecraft, in comparison to the other context files. Thus, it is quite important we use this file effectively.

We are given 3 DMOP files, each for different date ranges. As we will see in a bit, there is quite a jump in milliseconds between each of the files.

"ut_ms" is unix time, measuring the number of milliseconds since January 1st 1970 at 00:00:00 UTC. After doing some initial manipulations and EDA, it may be a good idea to convert this to datetime64, and set it as the index for the dataframe.

"subsystem" is a string. There are two types of string:
- Type (i) contain information about the subsystem and command that has been executed. They do not contain a full stop, e.g. "ASXX383C". The first 4 characters refer to a subsystem, and the remaining characters represent some command.
- Type (ii) represent flight dynamics events. They contain a "." in the middle of the string, e.g. "MAPO.000005". The first 4 characters refer to the event, and the remaining characters is an occurence number.
- There are also some other strings which do not fit either format, e.g. "Trigger" or "DISABTT_TRIGGERS"

The documentation does not explain what each code e.g. "ASXX" or "383C" or "MAPO" represents directly.

### Key observations
- The data is very clean, there are no null values
- The time granularity is not consistent at all
- There are considerable jumps in time between each of the 3 DMOP files, particularly from the 2nd to the 3rd
- The effects of these commands and events likely have a **delayed effect** on the overall problem, but I am not sure how this can be implemented effectively
- In the test DMOP dataset, the time granularity can be as small as 1 second, or as large as ~32.48 Earth days
- Many model solutions identify particular "pairs of commands" which are of particular importance. These pairs supposedly come as ON/OFF switches for something on the spacecraft, e.g. 'AAAAF40B0' and 'AAAAF40C0', or 'AAAAF40E0' and 'AAAAF40F0'. However, it is not explained how the authors found this out, or how you would implement these 'pairs' in an ML model.
- Even if we decide on a consistent time granularity we want to use for this project, it is not clear how we can include the information in the DMOP file. It is not as simple as having integer values that we can take averages of over the given time granularity. We have strings instead.

### Another key observation
This is completely separate to the DMOP file. The project description says that the power file has observations every 30/60 seconds. On closer look, the intervals are not exactly 30 or 60 seconds, but are slightly off by milliseconds. There are also much bigger intervals as well.

### General EDA

In [ ]:
# change these filepaths to suit yourself as needed
filepath1 = "train_set/context--2008-08-22_2010-07-10--dmop.csv"
filepath2 = "train_set/context--2010-07-10_2012-05-27--dmop.csv"
filepath3 = "train_set/context--2012-05-27_2014-04-14--dmop.csv"

# load the dataframes
dmop1 = pd.read_csv(filepath1)
dmop2 = pd.read_csv(filepath2)
dmop3 = pd.read_csv(filepath3)

In [ ]:
# find the average time granularity in the ut_ms columns
differences1 = dmop1['ut_ms'].diff()
average_diff1 = differences1.mean()

differences2 = dmop2['ut_ms'].diff()
average_diff2 = differences2.mean()

differences3 = dmop3['ut_ms'].diff()
average_diff3 = differences3.mean()

print("The average time granularity in the 3 files are " + str(average_diff1) + ", "
      + str(average_diff2) + " and " + str(average_diff3))

The average time granularity in the 3 files are 317203.13159652636, 405202.86367966846 and 361522.25617007265


In [ ]:
# this gives the jump in milliseconds from the 1st DMOP file to the 2nd DMOP file
int(dmop2.head(1)['ut_ms']) - int(dmop1.tail(1)['ut_ms'])

3516000

In [ ]:
# this gives the jump in milliseconds from the 2nd DMOP file to the 3rd DMOP file
int(dmop3.head(1)['ut_ms']) - int(dmop2.tail(1)['ut_ms'])

1157618000

In [ ]:
# merge the three separate DMOP files into one
dmop = pd.concat([dmop1,dmop2,dmop3], axis = 0)

In [ ]:
# just showing some examples of what the dataset looks like
dmop.sample(10)

,ut_ms,subsystem
137659,1266536394000,AAAAF23G1
102258,1254443110000,MOCS.0000003195
181621,1277232044000,APSF14A2
36975,1349407941000,AACFE05A
136662,1388884451000,AACFE03A
143538,1390893552000,AACFE91A
65188,1242848801000,AMMMF51A0
130332,1264368434000,AMMMF52D3
19585,1344131107000,AACFE91A
136775,1266335934000,AVVV02A0


In [ ]:
# check for any null values
dmop.isnull().sum()

ut_ms        0
subsystem    0
dtype: int64

In [ ]:
# new column containing 1s for type (i) strings (i.e. commands) and 0s for type (ii) strings (i.e. flight dynamics events)
dmop['command'] = (~ dmop['subsystem'].str.contains("\.")).astype(int)

# make a new column for the first 4 characters of each string
dmop['group'] = dmop['subsystem'].str[:4]

# make a new column for the remaining characters, removing any full stops
# contains commands for type (i) strings and occurence numbers for type (ii) strings
dmop['command_occurencenumber'] = dmop['subsystem'].str[4:]
dmop['command_occurencenumber'] = dmop['command_occurencenumber'].str.strip('.')

# give the counts of each type of command or flight dynamics event
counts = dmop.groupby(['group','subsystem'])['subsystem'].count().sort_values(ascending=False)
counts.head(400)

group  subsystem       
AAAA   AAAAF56A1           20035
AACF   AACFE91A            19493
APSF   APSF28A1            18312
AACF   AACFE03A            18227
APSF   APSF38A1            15611
AAAA   AAAAF20E1           13440
AVVV   AVVV02A0            10073
       AVVV03B0             9774
AAAA   AAAAF20C1            9205
AMMM   AMMMF10A0            8938
AACF   AACFE05A             8394
AAAA   AAAAF23G1            6450
       AAAAF57A1            5532
       AAAAF60A1            5530
AMMM   AMMMF18A0            5029
       AMMMF19A0            5004
APSF   APSF32A1             4893
       APSF50A2             4854
AACF   AACFM07A             4634
       AACFM06A             4600
AAAA   AAAAF93A1            4541
       AAAAF59A1            4480
APSF   APSF12H1             4076
ASSS   ASSSF06A0            3981
       ASSSF01A0            3980
AMMM   AMMMF23A0            3974
       AMMMF24A0            3962
AACF   AACFM01A             3792
       AACFM02A             3782
ATTT   ATTTF030A   

In [ ]:
# this is the dataframe we are left with
dmop.sample(12)

,ut_ms,subsystem,command,group,command_occurencenumber
56974,1240472637000,APSF28A1,1,APSF,28A1
51113,1238256873000,AMMMF51A0,1,AMMM,F51A0
108508,1257237159000,AAAAF20E1,1,AAAA,F20E1
34943,1290094494000,AMMMF18A0,1,AMMM,F18A0
130010,1264222087000,AAAAF93A1,1,AAAA,F93A1
12652,1223109033000,AVVV02A0,1,AVVV,02A0
13869,1342288180000,MOCS.0000005069,0,MOCS,0000005069
73700,1305039768000,MAPO.0000009390,0,MAPO,0000009390
59004,1357811988000,AACFE91A,1,AACF,E91A
44717,1293998762000,APSF50A2,1,APSF,50A2


In [ ]:
# look at the test DMOP file, change filepath if necessary
filepath_test = "test_set/context--2014-04-14_2016-03-01--dmop.csv"
dmop_test = pd.read_csv(filepath_test)

dmop_test.head(10)

,ut_ms,subsystem
0,1397433786000,ATTTF030A
1,1397434506000,AMMMF40A0
2,1397434515000,AACFE91A
3,1397434573000,ATTT309P
4,1397434575000,AACFM02A
5,1397434636000,AACFE03A
6,1397434638000,AACFE91A
7,1397434640000,AACFE05A
8,1397434761000,AMMMF10A0
9,1397434766000,AMMMF10A0


In [ ]:
# create a column which calculates the differences in milliseconds between each record
dmop_test['ut_ms_diff'] = dmop_test['ut_ms'].diff()
dmop_test.sample(12)

,ut_ms,subsystem,ut_ms_diff
104627,1438181258000,ATTT309P,58000.0
108201,1439823119000,AACFE05A,4000.0
60679,1418398494000,AMMMF40A0,635000.0
146651,1455304376000,AACFE91A,2000.0
21650,1404534397000,AACFE05A,3000.0
146968,1455430426000,AACFM02A,60000.0
141512,1453408142000,AACFE91A,8751000.0
43891,1412179288000,AACFE91A,5742000.0
142979,1453939745000,PENS.0000007846,3000.0
103649,1437751328000,AAAAF40J0,413000.0


In [ ]:
# observe the max and mins of these differences
diff_min = dmop_test['ut_ms_diff'].min()
diff_max = dmop_test['ut_ms_diff'].max()
print("Min time difference is " + str(diff_min) + ", max time difference is " + str(diff_max))

Min time difference is 1000.0, max time difference is 2806627000.0


In [ ]:
# 1000 ms is equal to 1 second
# 280662700 ms is equal to 32.48... Earth days

# clearly the time granularity in the test DMOP dataset is very inconsistent

In [ ]:
# back to the training DMOP file
# converting the ut_ms column to a datetime index
dmop['ut_ms'] = pd.to_datetime(dmop['ut_ms'], unit = 'ms')
dmop = dmop.set_index('ut_ms')

In [ ]:
dmop.sample(12)

,subsystem,command,group,command_occurencenumber
ut_ms,,,,
2014-03-16 04:45:10,AACFM07A,1,AACF,M07A
2013-01-21 02:40:27,AHHHF01P1,1,AHHH,F01P1
2011-05-25 00:38:38,APSF28A1,1,APSF,28A1
2012-11-17 03:11:38,AACFM06A,1,AACF,M06A
2009-12-21 18:44:06,AVVV02A0,1,AVVV,02A0
2010-10-05 01:37:44,AAAAF20E1,1,AAAA,F20E1
2011-06-25 13:50:37,AAAAF57A1,1,AAAA,F57A1
2009-09-20 01:11:08,AAAAF93A1,1,AAAA,F93A1
2010-05-02 17:35:25,AMMMF23A0,1,AMMM,F23A0


### Attempt to Identify Feature Importance

In [ ]:
# working with the power files
# reading in the power files
filepath_power1 = "train_set/power--2008-08-22_2010-07-10.csv"
filepath_power2 = "train_set/power--2010-07-10_2012-05-27.csv"
filepath_power3 = "train_set/power--2012-05-27_2014-04-14.csv"
power1 = pd.read_csv(filepath_power1)
power2 = pd.read_csv(filepath_power2)
power3 = pd.read_csv(filepath_power3)

# for now, we are not interested in trying to predict all 33 power values
# just do a few
columns_to_keep = ["ut_ms","NPWD2372","NPWD2401","NPWD2402","NPWD2451","NPWD2471"]
power1 = power1[columns_to_keep]
power2 = power2[columns_to_keep]
power3 = power3[columns_to_keep]

# merge these into one dataset
power = pd.concat([power1,power2,power3], axis = 0)

In [ ]:
# observing the time granularity of ut_ms in the power files
power['ut_ms_diff'] = power['ut_ms'].diff()
power_diff_min = power['ut_ms_diff'].min()
power_diff_max = power['ut_ms_diff'].max()
print("Min time difference is " + str(power_diff_min) + ", max time difference is " + str(power_diff_max))

Min time difference is 1.0, max time difference is 83167996.0


In [ ]:
# converting the ut_ms column to a datetime index
power['ut_ms'] = pd.to_datetime(power['ut_ms'], unit = 'ms')
power = power.set_index('ut_ms')

# drop the ut_ms_diff column, we will not need it
power = power.drop(columns = ['ut_ms_diff'])

In [ ]:
power.sample(12)

,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471
ut_ms,,,,,
2009-01-15 17:12:08.455,0.379161,0.001769,0.177440,0.006019,0.000857
2009-07-04 18:04:24.460,0.368583,0.001474,0.172173,0.005015,0.000714
2012-06-08 12:39:37.791,0.373872,0.001474,0.172173,0.005015,0.000714
2009-04-14 22:43:52.432,0.001821,0.001474,0.172173,0.006019,0.000714
2010-07-01 22:22:00.554,0.368583,0.001474,0.172173,0.005015,0.000714
2014-04-06 12:57:46.996,0.368583,0.001474,0.172173,0.005015,0.000714
2012-04-24 23:00:25.684,0.001821,0.001474,0.172173,0.005015,0.000714
2011-05-21 18:28:24.983,0.001821,0.001474,0.177440,0.005015,0.000714
2010-11-01 20:05:28.674,0.001821,0.001474,0.177440,2.155600,0.000714


In [ ]:
# set the index of the power dataset to exactly match the dmop dataset
power_reindexed = power.reindex(dmop.index)

# interpolate missing values
power_reindexed = power_reindexed.interpolate()

# backward fill any remaining missing values
power_reindexed = power_reindexed.bfill()

In [ ]:
X = dmop[['group', 'command']]
X_encoded = pd.get_dummies(X, prefix = 'group')
y1 = power_reindexed[['NPWD2372']]
y2 = power_reindexed[['NPWD2401']]
y3 = power_reindexed[['NPWD2402']]
y4 = power_reindexed[['NPWD2451']]
y5 = power_reindexed[['NPWD2471']]

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd

# Splitting the data into training and testing sets for y1
X_train_y1, X_test_y1, y_train_y1, y_test_y1 = train_test_split(X_encoded, y1, test_size=0.2, random_state=42)

# Splitting the data into training and testing sets for y2
X_train_y2, X_test_y2, y_train_y2, y_test_y2 = train_test_split(X_encoded, y2, test_size=0.2, random_state=42)

# Splitting the data into training and testing sets for y3
X_train_y3, X_test_y3, y_train_y3, y_test_y3 = train_test_split(X_encoded, y3, test_size=0.2, random_state=42)

# Splitting the data into training and testing sets for y4
X_train_y4, X_test_y4, y_train_y4, y_test_y4 = train_test_split(X_encoded, y4, test_size=0.2, random_state=42)

# Splitting the data into training and testing sets for y5
X_train_y5, X_test_y5, y_train_y5, y_test_y5 = train_test_split(X_encoded, y5, test_size=0.2, random_state=42)

# Ensure y_train_y1, y_train_y2, y_train_y3, y_train_y4, and y_train_y5 are 1-dimensional arrays
y_train_y1 = y_train_y1.values.ravel()
y_train_y2 = y_train_y2.values.ravel()
y_train_y3 = y_train_y3.values.ravel()
y_train_y4 = y_train_y4.values.ravel()
y_train_y5 = y_train_y5.values.ravel()

# Training the XGBoost model for y1
model_y1 = xgb.XGBRegressor(random_state=42)
model_y1.fit(X_train_y1, y_train_y1)

# Training the XGBoost model for y2
model_y2 = xgb.XGBRegressor(random_state=42)
model_y2.fit(X_train_y2, y_train_y2)

# Training the XGBoost model for y3
model_y3 = xgb.XGBRegressor(random_state=42)
model_y3.fit(X_train_y3, y_train_y3)

# Training the XGBoost model for y4
model_y4 = xgb.XGBRegressor(random_state=42)
model_y4.fit(X_train_y4, y_train_y4)

# Training the XGBoost model for y5
model_y5 = xgb.XGBRegressor(random_state=42)
model_y5.fit(X_train_y5, y_train_y5)

# Predicting the values of NPWD2372 for y1
y_pred_y1 = model_y1.predict(X_test_y1)

# Predicting the values of NPWD2401 for y2
y_pred_y2 = model_y2.predict(X_test_y2)

# Predicting the values of NPWD2402 for y3
y_pred_y3 = model_y3.predict(X_test_y3)

# Predicting the values of NPWD2403 for y4
y_pred_y4 = model_y4.predict(X_test_y4)

# Predicting the values of NPWD2404 for y5
y_pred_y5 = model_y5.predict(X_test_y5)

# Calculating model performance metrics for y1 (e.g., Mean Absolute Error)
from sklearn.metrics import mean_absolute_error
mae_y1 = mean_absolute_error(y_test_y1, y_pred_y1)
print("Mean Absolute Error for y1:", mae_y1)

# Calculating model performance metrics for y2 (e.g., Mean Absolute Error)
mae_y2 = mean_absolute_error(y_test_y2, y_pred_y2)
print("Mean Absolute Error for y2:", mae_y2)

# Calculating model performance metrics for y3 (e.g., Mean Absolute Error)
mae_y3 = mean_absolute_error(y_test_y3, y_pred_y3)
print("Mean Absolute Error for y3:", mae_y3)

# Calculating model performance metrics for y4 (e.g., Mean Absolute Error)
mae_y4 = mean_absolute_error(y_test_y4, y_pred_y4)
print("Mean Absolute Error for y4:", mae_y4)

# Calculating model performance metrics for y5 (e.g., Mean Absolute Error)
mae_y5 = mean_absolute_error(y_test_y5, y_pred_y5)
print("Mean Absolute Error for y5:", mae_y5)

# Getting feature importances for y1
importance_dict_y1 = model_y1.get_booster().get_score(importance_type='weight')

# Getting feature importances for y2
importance_dict_y2 = model_y2.get_booster().get_score(importance_type='weight')

# Getting feature importances for y3
importance_dict_y3 = model_y3.get_booster().get_score(importance_type='weight')

# Getting feature importances for y4
importance_dict_y4 = model_y4.get_booster().get_score(importance_type='weight')

# Getting feature importances for y5
importance_dict_y5 = model_y5.get_booster().get_score(importance_type='weight')

# Calculating the average importance of features across all datasets
combined_importance_dict = {}
for feature in set(list(importance_dict_y1.keys()) + list(importance_dict_y2.keys()) + list(importance_dict_y3.keys()) + list(importance_dict_y4.keys()) + list(importance_dict_y5.keys())):
    importance_y1 = importance_dict_y1.get(feature, 0)
    importance_y2 = importance_dict_y2.get(feature, 0)
    importance_y3 = importance_dict_y3.get(feature, 0)
    importance_y4 = importance_dict_y4.get(feature, 0)
    importance_y5 = importance_dict_y5.get(feature, 0)
    combined_importance_dict[feature] = (importance_y1 + importance_y2 + importance_y3 + importance_y4 + importance_y5) / 5

# Creating a DataFrame to display the average feature importances
importance_df_combined = pd.DataFrame({'Feature': list(combined_importance_dict.keys()), 'Average_Importance': list(combined_importance_dict.values())})
importance_df_combined = importance_df_combined.sort_values(by='Average_Importance', ascending=False)
print(importance_df_combined)


Mean Absolute Error for y1: 0.14054118796857806
Mean Absolute Error for y2: 0.00011469714298132523
Mean Absolute Error for y3: 2.040121092976889e-05
Mean Absolute Error for y4: 0.43676457363305166
Mean Absolute Error for y5: 4.89022820116449e-07
       Feature  Average_Importance
8   group_APWF                11.6
24  group_ADMC                11.0
3   group_AVVV                11.0
7   group_Trig                11.0
0   group_SCMN                10.8
21  group_OBCP                10.8
18  group_AXXX                10.6
1   group_ASEQ                10.4
23  group_AACF                10.4
31  group_ATMB                 9.8
25  group_DISA                 9.4
4   group_PDNE                 8.4
11  group_PDNS                 8.4
5   group_UPBE                 8.0
6   group_UPBS                 8.0
29  group_PPNS                 7.6
10  group_PPNE                 7.4
17  group_AHHH                 7.2
22  group_PENE                 7.2
27  group_AAAA                 7.2
28  group_ASXX     

### ON/OFF Switches
Some of the subsystem commands are likely to be ON/OFF switches for functions on the spacecraft. We know this through other literature on the project.

I spent a whole day trying to create code which can identify which subsystem commands these may be, to no success. The best I could do was find pairs of subsystem commands which differ by exactly one character. However, there are too many of these (over 600), so we cannot include all of them in our model. Any of these pairs could potentially be an ON/OFF switch. Without further actual literature, there is no way of knowing.

For now, I have used the switches found by other project attempts so we can move forward.

In [ ]:
## identifying subsystem commands which differ by exactly one character (potential ON/OFF switches)
# there are so many unique subsystem commands, that searching pairwise through them all would take too long
# ignore some of the commands, and use grouping, to increase efficiency


from collections import defaultdict

# most of the in-flight events only differ by a number
# we are not interested in these
# they all contain a full stop somewhere in the string, so ignore these
temp_dmop = dmop[~dmop['subsystem'].str.contains('\.')]

# group by the type of command to increase efficiency of this algorithm
grouped = temp_dmop.groupby('group')

# dictionary to store unique strings by group
strings_by_group = defaultdict(list)

for group_name, group_indices in grouped.groups.items():
    group_data = dmop.loc[group_indices]
    unique_strings = set(group_data['subsystem'])
    strings_by_group[group_name] = unique_strings

# function to check whether two strings differ by exactly one character
def differ_by_one(s1, s2):
    if len(s1) != len(s2):
        return False
    diff_count = sum(c1 != c2 for c1, c2 in zip(s1, s2))
    return diff_count == 1

# find the relevant pairs
result_pairs = []
for group, strings in strings_by_group.items():
    if len(strings) < 2:
        continue
    strings_list = list(strings)
    for i in range(len(strings_list)):
        for j in range(i+1, len(strings_list)):
            if differ_by_one(strings_list[i], strings_list[j]):
                result_pairs.append((strings_list[i], strings_list[j]))

# print results
print("Pairs of strings differing by one character:")
for pair in result_pairs:
    print(pair)

Pairs of strings differing by one character:
('AAAAF40M0', 'AAAAF40P0')
('AAAAF40M0', 'AAAAF40B0')
('AAAAF40M0', 'AAAAF40J0')
('AAAAF40M0', 'AAAAF40E0')
('AAAAF40M0', 'AAAAF40N0')
('AAAAF40M0', 'AAAAF40L0')
('AAAAF40M0', 'AAAAF40D0')
('AAAAF40M0', 'AAAAF40H0')
('AAAAF40M0', 'AAAAF40C0')
('AAAAF40M0', 'AAAAF40A0')
('AAAAF40M0', 'AAAAF40F0')
('AAAAF40M0', 'AAAAF40K0')
('AAAAF40M0', 'AAAAF40G0')
('AAAAF23B2', 'AAAAF23A2')
('AAAAF87A1', 'AAAAF57A1')
('AAAAF87A1', 'AAAAF85A1')
('AAAAF87A1', 'AAAAF83A1')
('AAAAF87A1', 'AAAAF84A1')
('AAAAF57A1', 'AAAAF56A1')
('AAAAF57A1', 'AAAAF59A1')
('AAAAF57A1', 'AAAAF58A1')
('AAAAF28B1', 'AAAAF26B1')
('AAAAF28B1', 'AAAAF20B1')
('AAAAF40P0', 'AAAAF40B0')
('AAAAF40P0', 'AAAAF40J0')
('AAAAF40P0', 'AAAAF40E0')
('AAAAF40P0', 'AAAAF40N0')
('AAAAF40P0', 'AAAAF40L0')
('AAAAF40P0', 'AAAAF40D0')
('AAAAF40P0', 'AAAAF40H0')
('AAAAF40P0', 'AAAAF40C0')
('AAAAF40P0', 'AAAAF40A0')
('AAAAF40P0', 'AAAAF40F0')
('AAAAF40P0', 'AAAAF40K0')
('AAAAF40P0', 'AAAAF40G0')
('AAAAF85A

The credit for most of the following work is Github user stephanos-stephani:

https://github.com/stephanos-stephani/MarsExpressChallenge/blob/master/dmopPairsExample.ipynb

In [ ]:
## pairs suspected to be important ON/OFF switches
pairs = [['AAAAF40B0','AAAAF40C0'],
        ['AAAAF40E0','AAAAF40F0'],
        ['AAAAF40D0','AAAAF40P0'],
        ['ASSSF01P0', 'ASSSF06P0'],
        ['AACFM01A','AACFM02A'],
        ['AACF325C','AACF325D'],
        ['AMMMF52D3','AMMMF52D4'],
        ['AMMMF18A0','AMMMF40A0'],
        ['AHHHF01P1','AHHHF50A2'],
        ['ATTTF030A', 'ATTTF030B'],
        ['ATTTF321P','ATTTF321R'],
        ['AACFM01A','AACFM02A'],
        ['AMMMF18A0','AMMMF19A0'],
        ['PENS','PENE'],
        ['MOCS','MOCE'],
         ['PDNS','PDNE'],
         ['PPNS','PPNE'],
         ['UPBS','UPBE']]

## count the number of occurences within each pair
for index, i in enumerate(pairs):
    print(index, i, (dmop['subsystem']==i[0]).sum(),(dmop['subsystem']==i[1]).sum())

0 ['AAAAF40B0', 'AAAAF40C0'] 2466 2453
1 ['AAAAF40E0', 'AAAAF40F0'] 1196 1196
2 ['AAAAF40D0', 'AAAAF40P0'] 791 86
3 ['ASSSF01P0', 'ASSSF06P0'] 2181 2178
4 ['AACFM01A', 'AACFM02A'] 3792 3782
5 ['AACF325C', 'AACF325D'] 3 3
6 ['AMMMF52D3', 'AMMMF52D4'] 1310 1310
7 ['AMMMF18A0', 'AMMMF40A0'] 5029 2232
8 ['AHHHF01P1', 'AHHHF50A2'] 676 1698
9 ['ATTTF030A', 'ATTTF030B'] 3693 3628
10 ['ATTTF321P', 'ATTTF321R'] 631 629
11 ['AACFM01A', 'AACFM02A'] 3792 3782
12 ['AMMMF18A0', 'AMMMF19A0'] 5029 5004
13 ['PENS', 'PENE'] 0 0
14 ['MOCS', 'MOCE'] 0 0
15 ['PDNS', 'PDNE'] 0 0
16 ['PPNS', 'PPNE'] 0 0
17 ['UPBS', 'UPBE'] 0 0


Some of the subsystem commands have similar numbers of occurences, reinforcing the possibility that they could be ON/OFF switches.

In [ ]:
## creating binary columns for each of these pairs
# call them pair0, pair1, pair2, and so on

## pair0

# initialise the column with NaN values
dmop['pair0'] = np.nan

# collect the indices where 'AAAAF40B0' appears as the ON switch
pair0_on_mask = (dmop['subsystem'] == "AAAAF40B0")
# same for the OFF switch, given by 'AAAAF40C0'
pair0_off_mask = (dmop['subsystem'] == 'AAAAF40C0')

# set the switch to ON where necessary
dmop.loc[pair0_on_mask, 'pair0'] = 1
# same for OFF
dmop.loc[pair0_off_mask, 'pair0'] = 0

# forward fill to propagate these values
dmop['pair0'] = dmop['pair0'].ffill()

# fill the NaN values which do not get filled at the start
dmop['pair0'].fillna(0, inplace = True)


## repeat the above with all other pairs in an automated fashion
for i in range(1,len(pairs)):
    column_name = 'pair' + str(i)
    dmop[column_name] = np.nan
    on_mask = (dmop['subsystem'] == pairs[i][0])
    off_mask = (dmop['subsystem'] == pairs[i][1])
    dmop.loc[on_mask, column_name] = 1
    dmop.loc[off_mask, column_name] = 0
    dmop[column_name] = dmop[column_name].ffill()
    dmop[column_name].fillna(0, inplace = True)

In [ ]:
dmop.head(50)

,subsystem,command,group,command_occurencenumber,pair0,pair1,pair2,pair3,pair4,pair5,...,pair8,pair9,pair10,pair11,pair12,pair13,pair14,pair15,pair16,pair17
ut_ms,,,,,,,,,,,,,,,,,,,,,
2008-08-22 00:00:11,AXXX301A,1,AXXX,301A,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:29,AAAAF20C1,1,AAAA,F20C1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:34,AAAAF57A1,1,AAAA,F57A1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:39,AAAAF23G1,1,AAAA,F23G1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:44,AAAAF60A1,1,AAAA,F60A1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:40:15,AXXX305A,1,AXXX,305A,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:47:15,AXXX380A,1,AXXX,380A,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:57:15,ASEQ4200,1,ASEQ,4200,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 01:09:41,ATTTF301E,1,ATTT,F301E,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Editing Final Features
There are some features which we will not need in the machine learning stage.

There are too many subsystem commands to be able to create dummy variables, so the subsystem column will be dropped. Same applies to the command_occurencenumber and group columns.

In [ ]:
dmop.drop(columns = ['subsystem', 'group', 'command_occurencenumber'], inplace = True)

In [ ]:
dmop.head()

,command,pair0,pair1,pair2,pair3,pair4,pair5,pair6,pair7,pair8,pair9,pair10,pair11,pair12,pair13,pair14,pair15,pair16,pair17
ut_ms,,,,,,,,,,,,,,,,,,,
2008-08-22 00:00:11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:29,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:34,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:39,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-22 00:28:44,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Resampling

We resample to an hourly time granularity. We also add a new feature, which counts the number of subsystem commands which occur within each hour. Hours with more subsystem commands occurring are likely more correlated with the power output of the spacecraft, compared to less busy hours.

In [ ]:
## resampling

# in order to resample, the datetime indices must be unique
# remove entries with duplicated datetime indices
dmop = dmop[~dmop.index.duplicated(keep='first')]

# resample to an hourly time granularity
dmop_resampled = dmop.resample('H').ffill()

# fill the NaNs at the very start
dmop_resampled = dmop_resampled.fillna(0)

In [ ]:
## creating the new feature

# calculating the needed values
entry_count_original = dmop.resample('H').count()

# adding this new column
dmop = pd.concat([dmop_resampled, entry_count_original.iloc[:,0]], axis = 1)

# rename the column
dmop.rename(columns={'command': 'no_of_subsystem_commands'}, inplace=True)

In [ ]:
dmop.head(50)

,no_of_subsystem_commands,pair0,pair1,pair2,pair3,pair4,pair5,pair6,pair7,pair8,pair9,pair10,pair11,pair12,pair13,pair14,pair15,pair16,pair17,no_of_subsystem_commands
ut_ms,,,,,,,,,,,,,,,,,,,,
2008-08-22 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
2008-08-22 01:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
2008-08-22 02:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
2008-08-22 03:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
2008-08-22 04:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2008-08-22 05:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2008-08-22 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2008-08-22 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
2008-08-22 08:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56


### Start and End Times
We want to make sure that all context files start and end at exactly the same datetime, so we can merge them. It helps to look at the data we will be using to build the model.

In [ ]:
## start datetime
power.head()

,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471
ut_ms,,,,,
2008-08-22 00:00:09.542,0.001821,0.001474,0.172173,0.005015,0.000714
2008-08-22 00:00:41.537,0.002185,0.001769,0.177440,0.006019,0.000714
2008-08-22 00:01:13.542,0.002185,0.001474,0.172173,0.005015,0.000571
2008-08-22 00:01:45.537,0.001821,0.001474,0.172173,0.005015,0.000714
2008-08-22 00:02:17.542,0.001821,0.001769,0.177440,0.005015,0.000714


In [ ]:
## end datetime
power.tail()

,NPWD2372,NPWD2401,NPWD2402,NPWD2451,NPWD2471
ut_ms,,,,,
2014-04-13 23:57:31.007,0.001821,0.001474,0.172173,0.005015,0.000714
2014-04-13 23:58:03.014,0.001821,0.001474,0.177440,0.005015,0.000714
2014-04-13 23:58:35.024,0.001821,0.001474,0.172173,0.005015,0.000714
2014-04-13 23:59:07.014,0.001821,0.001474,0.172173,0.005015,0.000714
2014-04-13 23:59:39.005,0.001457,0.001474,0.172173,0.005015,0.000714


### Saving the CSV

In [ ]:
## save the pre-processed dataset as a .csv file, ready for machine learning
# ensure index parameter is set to True
dmop.to_csv("dmop.csv", sep=',', index=True, encoding='utf-8')